In [1]:
!pip install --upgrade albumentations

     |████████████████████████████████| 102 kB 13.2 MB/s 
     |████████████████████████████████| 47.6 MB 2.2 MB/s 
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [4]:
import numpy as np
import matplotlib.pyplot as plt

import albumentations as A
from albumentations.pytorch import ToTensorV2

import torch 
from torchvision import datasets, transforms
from torchsummary import summary

In [6]:
from albumentations.pytorch import ToTensorV2

from google.colab import drive
drive.mount('/content/drive')

BASE_DIR='/content/drive/My Drive/Session_8'
%cd $BASE_DIR

Mounted at /content/drive
/content/drive/My Drive/Session_8


In [50]:
from models import resnet
from utils import gradcam
from dataloaders.CIFAR10 import download_data, get_train_test_loaders,LoadDataset
from main import fit_model

In [9]:
import importlib
importlib.reload(gradcam)
# importlib.reload(resnet)

<module 'utils.gradcam' from '/content/drive/My Drive/Session_8/utils/gradcam.py'>

In [18]:
# Apply these transforms while training:
# RandomCrop(32, padding=4)
# CutOut(16x16)
# Rotate(±5°)

train_transform = A.Compose({
  A.Rotate (limit=5, interpolation=1, border_mode=4, value=None, mask_value=None, always_apply=False, p=0.5),
  A.Sequential([A.CropAndPad(px=4, keep_size=False), #padding of 2, keep_size=True by default
                A.RandomCrop(32,32)]),
  A.CoarseDropout(1, 16, 16, 1, 16, 16,fill_value=0.473363, mask_fill_value=None),
  A.Normalize((0.49139968, 0.48215841, 0.44653091), (0.24703223, 0.24348513, 0.26158784)),
})

test_transform = A.Compose({
  A.Normalize((0.49139968, 0.48215841, 0.44653091), (0.24703223, 0.24348513, 0.26158784))
})

In [19]:
cuda = torch.cuda.is_available()
print("CUDA Available:", cuda)
SEED = 1
# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)
    BATCH_SIZE=64
else:
    BATCH_SIZE=32

CUDA Available: True


In [28]:
import cv2
import torchvision
import torch
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

torch.manual_seed(2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
cv2.setNumThreads(0)
cv2.ocl.setUseOpenCL(False)

class Cifar10SearchDataset(torchvision.datasets.CIFAR10):
    def __init__(self, root="~/data/cifar10", train=True, download=True, transform=None):
        super().__init__(root=root, train=train, download=download, transform=transform)

    def __getitem__(self, index):
        image, label = self.data[index], self.targets[index]

        if self.transform is not None:
            transformed = self.transform(image=image)
            image = transformed["image"]

        return image, label


train_transforms = A.Compose(
    [
        A.HorizontalFlip(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=45, p=0.5),
        A.CoarseDropout(max_holes = 1, max_height=16, max_width=16, min_holes = 1, min_height=16, min_width=16,
                        fill_value=0.4734),
        A.Normalize(
            mean = (0.4914, 0.4822, 0.4465),
            std = (0.2470, 0.2435, 0.2616),
            p =1.0
        ),
        ToTensorV2()
    ],
    p=1.0
)

test_transforms = A.Compose(
    [
        A.Normalize(
            mean = (0.4914, 0.4822, 0.4465),
            std = (0.2470, 0.2435, 0.2616),
            p =1.0
        ),
        ToTensorV2()
    ]
)

class args():
    def __init__(self,device = 'cpu' ,use_cuda = False) -> None:
        self.batch_size = 128
        self.device = device
        self.use_cuda = use_cuda
        self.kwargs = {'num_workers': 2, 'pin_memory': True} if self.use_cuda else {}

trainset = Cifar10SearchDataset(root='./data', train=True,
                                        download=True, transform=train_transforms)
                                        
trainloader = torch.utils.data.DataLoader(trainset, batch_size=args().batch_size,
                                          shuffle=True, **args(use_cuda=True).kwargs)


testset = Cifar10SearchDataset(root='./data', train=False,
                                       download=True, transform=test_transforms)

testloader = torch.utils.data.DataLoader(testset, batch_size=args().batch_size,
                                         shuffle=False, **args(use_cuda=True).kwargs)

##
# functions to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))


# # get some random training images
# dataiter = iter(trainloader)
# images, labels = dataiter.next()

# # show images
# imshow(torchvision.utils.make_grid(images))
# # print labels
# print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

def get_statistics(dataset):

    """
    input = dataset should be a of type torchvision.datasets calculate statistics of training data only 
    Calculated stats are->
    torch.Size([3, 32, 32, 50000])
    Mean =  tensor([0.4914, 0.4822, 0.4465])
    Standard deviation =  tensor([0.2470, 0.2435, 0.2616])
    
    """
    imgs = torch.stack([img_t for img_t,_ in dataset],dim = 3)
    print(imgs.shape)

    mean_calc = imgs.view(3,-1).mean(dim = 1)
    std_dev_calc = imgs.view(3,-1).std(dim = 1)

    print('Mean = ',mean_calc)
    print('Standard deviation = ',std_dev_calc)

    return mean_calc,std_dev_calc

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [57]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cuda


In [30]:
net_18 = resnet.ResNet18().to(device)
summary(net_18, input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 64, 32, 32]          36,864
       BatchNorm2d-4           [-1, 64, 32, 32]             128
            Conv2d-5           [-1, 64, 32, 32]          36,864
       BatchNorm2d-6           [-1, 64, 32, 32]             128
        BasicBlock-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
       BatchNorm2d-9           [-1, 64, 32, 32]             128
           Conv2d-10           [-1, 64, 32, 32]          36,864
      BatchNorm2d-11           [-1, 64, 32, 32]             128
       BasicBlock-12           [-1, 64, 32, 32]               0
           Conv2d-13          [-1, 128, 16, 16]          73,728
      BatchNorm2d-14          [-1, 128,

In [ ]:
optimizer = torch.optim.SGD(net_18.parameters(), lr=0.01, momentum=0.9)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,mode='min', patience=8, verbose=True)

net_18, history_18 = fit_model(
    net=net_18,
    train_loader=trainloader, test_loader=testloader,
    scheduler=scheduler, 
    optimizer=optimizer, device=device, NUM_EPOCHS=40
)